#  LangChain-Rag实战
参考LangChain官方文档实现


## 环境准备
- conda 分环境管理python环境和包
```
## 查看当前conda 存在那些环境
conda env list
# conda environments:
#
base                  *  /home/gengzi/anaconda3
env-vllm                 /home/gengzi/anaconda3/envs/env-vllm

## 切换到需要运行的环境
conda activate env-vllm
```
- .env 创建一个env配置文件


In [46]:
from dotenv import load_dotenv
import os
# 获取.env 中配置
env = load_dotenv()
API_KEY = os.getenv("API_KEY")

步骤1： 将原始资料整理，拆分成一块一块内容，进行embedding，转化为词向量，存入向量空间中




In [47]:
import bs4
from langchain_community.document_loaders import WebBaseLoader

page_url = "https://wangchujiang.com/linux-command/c/cd.html"

loader = WebBaseLoader(web_paths=[page_url],
                       autoset_encoding = False,
                       encoding = "utf-8",
                       bs_get_text_kwargs = {
                                "separator": " ",
                                "strip": True
                            }
                       )
docs = []
async for doc in loader.alazy_load():
    docs.append(doc)
assert len(docs) == 1
doc = docs[0]
print(doc)


page_content='cd 命令，Linux cd 命令详解：切换用户当前工作目录。 -  Linux 命令搜索引擎 logo Linux Command Logo. https://github.com/jaywcjlove 搜索 纠正错误 添加实例 cd 切换用户当前工作目录。 概要 cd [ -L | [ -P [ -e ] ] ] [ dir ] 主要用途 切换工作目录至 dir 。其中 dir 的表示法可以是绝对路径或相对路径。 若参数 dir 省略，则默认为使用者的shell变量 HOME 。 如果 dir 指定为 ~ 时表示为使用者的shell变量 HOME ， . 表示当前目录， .. 表示当前目录的上一级目录。 环境变量 CDPATH 是由冒号分割的一到多个目录，你可以将常去的目录的上一级加入到 CDPATH 以便方便访问它们；如果 dir 以 / 开头那么 CDPATH 不会被使用。 当 shopt 选项 cdable_vars 打开时，如果 dir 在 CDPATH 及当前目录下均不存在，那么会把它当作变量，读取它的值作为要进入的目录。 参数 dir（可选）：指定要切换到的目录。 选项 -L （默认值）如果要切换到的目标目录是一个符号连接，那么切换到符号连接的目录。 -P 如果要切换到的目标目录是一个符号连接，那么切换到它指向的物理位置目录。 -  当前工作目录将被切换到环境变量 OLDPWD 所表示的目录，也就是前一个工作目录。 返回值 返回状态为成功除非无法进入指定的目录。 例子 cd # 进入用户主目录； cd / # 进入根目录 cd ~ # 进入用户主目录； cd .. # 返回上级目录（若当前目录为“/“，则执行完后还在“/"；".."为上级目录的意思）； cd .. / .. # 返回上两级目录； cd ! $ # 把上个命令的参数作为cd参数使用。 关于切换到上一个工作目录的说明 cd - # 命令会首先显示要切换到的目标目录，然后再进入。 cd ${ OLDPWD } # 命令会直接切换到上一个工作目录。 关于 CDPATH # 设置桌面文件夹作为CDPATH的值。 CDPATH = '~/Desktop' # 假设我们接下来要演示涉及到的路径~和~/Desktop下没有test3文件夹，现在新建它们。 mkdir ~/

In [48]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,  # chunk size (characters)
    chunk_overlap=5,  # chunk overlap (characters)
    add_start_index=True,  # track index in original document
)
all_splits = text_splitter.split_documents(docs)

print(f"Split blog post into {len(all_splits)} sub-documents.")
for split in all_splits:
    print(split.page_content)
    print("----" )

Split blog post into 9 sub-documents.
cd 命令，Linux cd 命令详解：切换用户当前工作目录。 -  Linux 命令搜索引擎 logo Linux Command Logo. https://github.com/jaywcjlove 搜索 纠正错误 添加实例 cd 切换用户当前工作目录。 概要 cd [ -L | [ -P [ -e ] ] ] [ dir ] 主要用途 切换工作目录至 dir 。其中 dir
----
dir 的表示法可以是绝对路径或相对路径。 若参数 dir 省略，则默认为使用者的shell变量 HOME 。 如果 dir 指定为 ~ 时表示为使用者的shell变量 HOME ， . 表示当前目录， .. 表示当前目录的上一级目录。 环境变量 CDPATH 是由冒号分割的一到多个目录，你可以将常去的目录的上一级加入到 CDPATH 以便方便访问它们；如果 dir 以 / 开头那么
----
开头那么 CDPATH 不会被使用。 当 shopt 选项 cdable_vars 打开时，如果 dir 在 CDPATH 及当前目录下均不存在，那么会把它当作变量，读取它的值作为要进入的目录。 参数 dir（可选）：指定要切换到的目录。 选项 -L （默认值）如果要切换到的目标目录是一个符号连接，那么切换到符号连接的目录。 -P
----
-P 如果要切换到的目标目录是一个符号连接，那么切换到它指向的物理位置目录。 -  当前工作目录将被切换到环境变量 OLDPWD 所表示的目录，也就是前一个工作目录。 返回值 返回状态为成功除非无法进入指定的目录。 例子 cd # 进入用户主目录； cd / # 进入根目录 cd ~ # 进入用户主目录； cd .. #
----
.. # 返回上级目录（若当前目录为“/“，则执行完后还在“/"；".."为上级目录的意思）； cd .. / .. # 返回上两级目录； cd ! $ # 把上个命令的参数作为cd参数使用。 关于切换到上一个工作目录的说明 cd - # 命令会首先显示要切换到的目标目录，然后再进入。 cd ${ OLDPWD } # 命令会直接切换到上一个工作目录。 关于 CDPATH #
----
# 设置桌面文件夹作为CDPATH的值。 CDPATH = '

In [49]:
from langchain_community.embeddings import ZhipuAIEmbeddings

embeddings = ZhipuAIEmbeddings(
    model="embedding-3",
    api_key= API_KEY
    # With the `embedding-3` class
    # of models, you can specify the size
    # of the embeddings you want returned.
    # dimensions=1024
)

emb_docs =[]

for split in all_splits:
    embedding_docs = embeddings.embed_documents(split.page_content)
    emb_docs.append(embedding_docs)
    print(embedding_docs)



[[-0.005726217, -0.019136457, 0.041688036, 0.026359243, -0.031266026, 0.030500568, -0.0012138156, 0.001787296, 0.019303288, -0.015397488, 0.014435759, -0.031835213, 0.0083807865, -0.005421996, -0.013866572, 0.0019737538, -0.007291481, -0.0009893304, -0.03707566, -0.019735085, -0.00654565, -0.0020240485, -0.010044186, 0.01254174, -0.014475012, -0.003385681, 0.0050981487, -0.011766468, 0.011187468, -0.024161004, 0.004143779, 0.10873433, 0.006251243, -0.01733076, -0.00059494755, -0.04133475, -0.0017038807, 0.0023859236, 0.0044529065, 0.00045357083, 0.010883247, 0.01212957, 0.0037904906, -0.007065769, -0.043964785, 0.005000013, -0.035132572, 0.0036310202, -0.010088348, 0.030873485, -0.027576126, -0.015417115, -0.006015717, -0.0005342261, 0.010745857, 0.01829249, 0.014337623, 0.014759606, -0.0076202354, -0.03185484, -0.047890212, 0.008861652, 0.023631072, 0.004663898, 0.020333713, 0.023120767, -0.026614396, -0.022924494, -0.046830345, -0.003510804, 0.015220844, 0.07458311, -0.024906835, 0.0

In [62]:
from langchain_chroma import Chroma


vector_store = Chroma(
    collection_name="web_documents",
    embedding_function=embeddings,
    persist_directory="./chroma_langchain_db",  # Where to save data locally, remove if not necessary
)

vector_store.add_documents(all_splits)




TypeError: VectorStore.add_documents() takes 2 positional arguments but 3 were given

步骤2： 接受用户提问，将问题也embedding后，查询向量数据库获取相似答案



In [51]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

example_messages = prompt.invoke(
    {"context": "(context goes here)", "question": "(question goes here)"}
).to_messages()

assert len(example_messages) == 1
print(example_messages[0].content)

/home/gengzi/anaconda3/envs/env-vllm/lib/python3.12/site-packages/langsmith/client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: (question goes here) 
Context: (context goes here) 
Answer:


In [52]:
from langchain_community.chat_models import ChatZhipuAI
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

llm = ChatZhipuAI(
    model="glm-4",
    temperature=0.5,
    api_key=API_KEY,
)

In [93]:
from langchain_core.documents import Document
from typing_extensions import List, TypedDict


class State(TypedDict):
    question: str
    context: List[Document]
    answer: str

def retrieve(state: State):
    ## 这个评分是一个距离，越接近0，就越相似
    retrieved_docs = vector_store.similarity_search_with_score(state["question"],k=5)
    filtered_docs = [(doc, score) for doc, score in retrieved_docs if score <= 0.9]
    return {"context": filtered_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc,score in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})

    response = llm.invoke(messages)
    return {"answer": response.content}

步骤3：获取到的相似内容，通过大模型优化输出


In [97]:
from langgraph.graph import START, StateGraph

graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

result = graph.invoke({"question": "今天的天气?"})

print(f'Context: {result["context"]}\n\n')
print(f'Answer: {result["answer"]}')

Context: []


Answer: 抱歉，您询问的“今天的天气”需要实时的天气信息，而提供的上下文中并没有包含相关数据。我无法给出具体的回答，建议查询当地的天气预报获取准确信息。


In [58]:
vector_store.delete_collection()

In [73]:

all = vector_store.get(ids=["ce08ffd8-aad3-4137-9ba0-6dc12c842bc9"],include=["embeddings"])
print(all)

# any = vector_store.get_by_ids(["ce08ffd8-aad3-4137-9ba0-6dc12c842bc9"])
# print(any)

{'ids': ['ce08ffd8-aad3-4137-9ba0-6dc12c842bc9'], 'embeddings': array([[-0.00572622, -0.01913646,  0.04168804, ...,  0.0105594 ,
        -0.02276748,  0.00871935]]), 'documents': None, 'uris': None, 'data': None, 'metadatas': None, 'included': [<IncludeEnum.embeddings: 'embeddings'>]}
